<a href="https://colab.research.google.com/github/Eandrushenko/CPTS-437/blob/master/HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Elijah Andrushenko
CPTS 437
1-17-2019
Introduction to Machine Learning

1. 10 Features
1) Font Color - One font is grey while the other is black.
2) Font Size - The sizes are slightly different and how big each letter is relative to the other letters differ.
3) Character Spacing - The spacing for my letters are very close compared to Sam's letter spacing.
4) The Os - My Os are closer to an ellipse while Sam's are similiar to a circle.
5) The Us - My Us have a little part at the end while Sam.s are more like a horseshoe.
6) the Gs - My Gs are similiar to cursive Gs, and Sam's are simpler.
7) Smoothness - My font type is smoother while Sam's are more blocky.
8) Word Spacing - The Spacing from one word to another for me are closer compared to Sam's word spacing.
9) Character Thickness - My letters are thicker than Sam's letters.
10) The Ss - My Ss are squiggly while Sam's tend to be blocky and some look like 5s.

2. See Photo Attachment--> (http://prntscr.com/m8pucx)

3. Entropy(3, 3) = 1

F1 = (1/3)(Entropy(1,1))+(1/3)(Entropy(2, 0))+(1/3)(Entropy(0, 2)) = 1/3
F1 = 1 - 1/3 = 2/3 = 0.67

F2 = (1/2)(Entropy(2, 1))+(1/2)(Entropy(1, 2))+0(Entropy(0,0)) = 0.92
F2 = 1 - 0.92 = 0.08

F3 = (1/3)(Entropy(1, 1))+0(Entropy(0, 0))+(2/3)(Entropy(2,2)) = 1
F3 = 1 - 1 = 0

F1 would be the chosen root of the decision tree since it has the highest information gain at 0.67.



4. To determine if a decision tree has over fit for its given training data we can calculate the ratio of (test data / training data) accuracy, at any size of the tree. The closer to 1 this ratio is, then the more fit the training data is with the test data. The further from 1 this ratio is, then the more overfit it is for the training data.
Based on the graph and my explanation above, The optimal size decision tree size would be 10, since at 10 nodes we have the highest accuracy for our test data and the ratio at 10 nodes would be (0.75 accuracy for test data/ 0.75 accuracy for training data) = 1, so it is a perfect fit, At higher numbers of nodes our test accuracy decreases and our ratio diverges away from 1.

5. I wasn't sure how to modify your code, even took an extra day to try to figure it out no luck....I was going to start from scratch but theres no time at this point. If the test data was more than 50% accurate then it is better than random guess since there is a 50/50 shot at getting the answer right, so if it does better than that then it must be better.

In [0]:
#SETUP
#Code taken from Dr. Cook to upload har.csv
import math
import collections
import numpy
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

# Read a data file in csv format, separate into features and class arrays
def read_data():
  data = numpy.loadtxt(fname='/content/gdrive/My Drive/Colab Notebooks/har.csv', delimiter=',')
  X = data[:,:-1] # features are all values but the last on the line
  y = data[:,-1] # class is the last value on the line
  return X, y

data, answer = read_data()
  

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#Part 1, replacing gini measure with Information Gain Measure
#This was also provided by Dr. Cook
def Entropy(pos, neg):
  pf = pos/float(pos + neg)
  nf = neg/float(pos + neg)
  if (pf == 0):
    term1 = 0
  else:
    term1 = -pf * math.log(pf, 2.0)
  if (nf == 0):
    term2 = 0
  else:
    term2 = -nf * math.log(nf, 2.0)
  entropy = term1 + term2
  return entropy

def Gain(pos, neg, splits):
  start = Entropy(pos, neg)
  #print(start)
  mysum = start
  for feature in splits:
    #print(feature)
    size = (feature[0] + feature[1]) / (pos+neg)
    feature_entropy = Entropy(feature[0], feature[1])
    #print(feature_entropy)
    mysum -= size*feature_entropy
    return mysum
  
#print(Entropy(3, 3))
#print(Gain(3, 3, [[1, 1], [0, 2], [2, 0]]))

# Calculate the Gini index for a subset of the dataset
def gini_index(groups, classes):
   # count all samples at split point
   num_instances = float(sum([len(group) for group in groups]))

   gini = 0.0 # sum weighted Gini index for each group
   for group in groups:
      size = float(len(group))
      if size == 0: # avoid divide by zero
         continue
      score = 0.0
      # score the group based on the score for each class
      for class_val in classes:
         p = [row[-1] for row in group].count(class_val) / size
         score += p * p
      # weight the group score by its relative size
      gini += (1.0 - score) * (size / num_instances)
   return gini


In [0]:
# Create child splits for a node or make a leaf node
def split(node, max_depth, depth):
   left, right = node['groups']
   del(node['groups'])
   # check for a no split
   if not left or not right:
      node['left'] = node['right'] = create_leaf(left + right)
      return
   # check for max depth
   if depth >= max_depth:
      node['left'], node['right'] = create_leaf(left), create_leaf(right)
      return
   node['left'] = select_attribute(left)
   split(node['left'], max_depth, depth+1)
   node['right'] = select_attribute(right)
   split(node['right'], max_depth, depth+1)
    


# split the dataset based on an attribute and attribute value
def test_split(index, value, dataset):
   left, right = list(), list()
   for row in dataset:
      if row[index] < value:
         left.append(row)
      else:
         right.append(row)
   return left, right


# Select the best split point for a dataset
def select_attribute(dataset):
   class_values = list(set(row[-1] for row in dataset))
   b_index, b_value, b_score, b_groups = 999, 999, 999, None
   for index in range(len(dataset[0])-1):
      for row in dataset:
         groups = test_split(index, row[index], dataset)
         gini = gini_index(groups, class_values)
         if gini < b_score:
            b_index, b_value, b_score, b_groups = index, row[index], gini, groups
   return {'index':b_index, 'value':b_value, 'groups':b_groups}



In [0]:
# Create a leaf node class value
def create_leaf(group):
   outcomes = [row[-1] for row in group]
   return max(set(outcomes), key=outcomes.count)

In [0]:
# Build a decision tree
def build_tree(train, max_depth):
   root = select_attribute(train)
   split(root, max_depth, 1)
   return root
  
  
# Print a decision tree
def print_tree(node, depth=0):
   if depth == 0:
      print('Tree:')
   if isinstance(node, dict):
      print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
      print_tree(node['left'], depth+1)
      print_tree(node['right'], depth+1)
   else:
      print('%s[%s]' % ((depth*' ', node)))
      
      
  
if __name__ == "__main__":
   dataset = [[2.771244718,1.784783929,0], [1.728571309,1.169761413,0],
              [3.678319846,2.812813570,0], [3.961043357,2.619950320,0],
              [2.999208922,2.209014212,0], [7.497545867,3.162953546,1],
              [9.00220326, 3.339047188,1], [7.444542326,0.476683375,1],
              [10.12493903,3.234550982,1], [6.642287351,3.319983761,1]]
   tree = build_tree(dataset, 1)
   print_tree(tree)

Tree:
[X1 < 6.642]
 [0]
 [1]


In [0]:
# Make a prediction with a decision tree
def predict(node, row):
   if row[node['index']] < node['value']:
      if isinstance(node['left'], dict):
         return predict(node['left'], row)
      else:
         return node['left']
   else:
      if isinstance(node['right'], dict):
         return predict(node['right'], row)
      else:
         return node['right']

        
if __name__ == "__main__":
   dataset = [[2.771244718,1.784783929,0], [1.728571309,1.169761413,0],
              [3.678319846,2.812813570,0], [3.961043357,2.619950320,0],
              [2.999208922,2.209014212,0], [7.497545867,3.162953546,1],
              [9.00220326, 3.339047188,1], [7.444542326,0.476683375,1],
              [10.12493903,3.234550982,1], [6.642287351,3.319983761,1]]
   tree = build_tree(dataset, 3)
   print_tree(tree)
   for row in dataset:
      prediction = predict(tree, row)
      print('Predicted=%d, Ground truth=%d' % (prediction, row[-1]))

Tree:
[X1 < 6.642]
 [X1 < 2.771]
  [X1 < 1.729]
   [0]
   [0]
  [X1 < 2.771]
   [0]
   [0]
 [X1 < 7.498]
  [X1 < 7.445]
   [1]
   [1]
  [X1 < 7.498]
   [1]
   [1]
Predicted=0, Ground truth=0
Predicted=0, Ground truth=0
Predicted=0, Ground truth=0
Predicted=0, Ground truth=0
Predicted=0, Ground truth=0
Predicted=1, Ground truth=1
Predicted=1, Ground truth=1
Predicted=1, Ground truth=1
Predicted=1, Ground truth=1
Predicted=1, Ground truth=1
